# Exercise 1

Decypher the following string that has been encrypted using the Vigenère code:

In [1]:
c = "MZMTELFMPLWUFEWXTZPCJDQPBVUKSIEEQDIMIIDRLQNTPWFBZNYNTTQLMAFAGQADDYXOPLIDIWYXFIN"\
    "ISZBSCZUOPLIDMNGTTMCCEDTTCVMBEYGWACCPUMOMRWVZUPQLRCSRBSCTXITLXQFEGSDCDCSRICCGAO"\
    "YGDMJWCAAZOYWMSPWOMATQOUAXCXTWOFEPVZQYOPOCTQVOCROQPQOMATQOUELQXTMQGVEBEMTGJWGWT"\
    "IYYGOWFLXANEFIMBEYGWJFRMFQDAPQICRLMBEFIDMHCVQWEFIDAHFSIMCCEIICCSRQEGRQQRFXQMYDM"\
    "RBJDSGZNFEDTPQFMJMYKQELQKAIOCHUVEMFDMLIMZOEFIHQRCRQZPAMBPPPATMYHSTVSYPXJCMGWBSU"\
    "EUBPQWGJXGXFMOYRQENGTTMCRSFPPHSGZYYPANEFIEWNGIFGZDXTMLPXEESCRNIMZESMDFSIMORLMBE"\
    "FAMQECWOQAFIDELQIEAPLXUIWJCVCDREZWEFIDZPAVQIEGSZWQRLQDTEIZMCCGUXSCVFPHYMFMDALMT"\
    "WCRSMOZENJLEIFWMPIMSSGWOQAFIDMYASPMORAUKPUMFPVCCEWQBMRNPPIZBWCRSBSZENJLEIECNAIQ"\
    "LPBMZLPAVKXEGRSIDYQBTPULUKSRYDVPBSGBEMFQBSCTAMXRLQDTQMAVZDWUVMWEXNCCHFMYLCEWYCR"\
    "OZJNXQLLAGAZOGRSBZRLQSPWAAZOCQUTJRLQNTPWFVLKIANECRZGDMREETDINIMZESMYCZQZPVTXITL"\
    "IPBSCQQBSMHTMFQIPAESHUMDMJNIMZESMDLSFMDPIHMLJXTIEFITIOSWQLEFIYMEFSPTLRIDXFZPUAS"\
    "CHNGVYWUAVGEZLDSKSMDRXTIEFITIOZIQVFQMZOEFIYMEFSPIDCEDTJYWQQRFXQMYDSGZEWWUF"

## 1. Determine length of key

For each trigram in `c`, calculate a list that contains all indices where that trigram occurs in `c`:

In [2]:
from collections import defaultdict

trigram_indices = defaultdict(list)
for i in range(len(c)-3):
    trigram = c[i:i+3]  # trigram = [c[i], c[i+1], c[i+2]]
    trigram_indices[trigram].append(i)
    
# print all trigrams (and corresponding indices) that occur at least 4 times in `c`
filtered_trigram_indices = {key:value for key, value in trigram_indices.items() if len(value) > 3}
sorted(filtered_trigram_indices.items())

[('BSC', [82, 132, 672, 792]),
 ('EFI', [248, 273, 283, 358, 433, 508, 838, 848, 893, 908]),
 ('FID', [274, 284, 484, 509, 579]),
 ('IMZ', [354, 457, 772, 817]),
 ('MBE', [106, 251, 271, 471]),
 ('RLQ', [39, 524, 679, 729, 744])]

For each trigram, calculate all index differences between two occurances of the same trigram:

In [3]:
from itertools import combinations

idx_differences = []
for key, indices in trigram_indices.items():
    idx_differences.extend(second - first for first, second in combinations(indices, 2))

For each possible key length `n`, calculate the number of index differences that are a multiple of `n`:

In [4]:
matches = []
for n in range(2, 11):
    num_matches = sum(d%n==0 for d in idx_differences)
    matches.append((n, num_matches))
    
# key length is the length that has the most matches
key_length, num_matches = max(matches, key=lambda x: x[1])
print(f"Key length: {key_length}")

Key length: 5


# 2. Calculate the key

Next, we need to calculate the key using a character frequency analysis.

Partition the input `c` into `key_length` parts:

In [5]:
partitions = [c[i::key_length] for i in range(key_length)]

We are assuming that the text is written in english which leads to the following distribution of characters (Source: [Wikipedia](https://en.wikipedia.org/wiki/Letter_frequency#Relative_frequencies_of_letters_in_the_English_language)):

In [6]:
char_frequencies = {'A': 8.167, 'B': 1.492, 'C': 2.782, 'D': 4.253, 'E': 12.702,
                    'F': 2.228, 'G': 2.015, 'H': 6.094, 'I': 6.966, 'J': 0.153,
                    'K': 0.772, 'L': 4.025, 'M': 2.406, 'N': 6.749, 'O': 7.507,
                    'P': 1.929, 'Q': 0.095, 'R': 5.987, 'S': 6.327, 'T': 9.056,
                    'U': 2.758, 'V': 0.978, 'W': 2.360, 'X': 0.150, 'Y': 1.974,
                    'Z': 0.074}
en_char_freq_list = list(char_frequencies.values())

In [7]:
from collections import Counter, defaultdict

partition_frequencies = []
for partition in partitions:
    char_counts = defaultdict(int, Counter(partition))
    partition_freq_list = [char_counts[i] / len(partition) for i in char_frequencies.keys()]
    partition_frequencies.append(partition_freq_list)

Find best matching shift:

In [8]:
offset_values = []

for p_freq in partition_frequencies:
    deviations = []
    for offset in range(26):
        deviation = 0
        for char_idx in range(26):
            deviation += (p_freq[(char_idx + offset)%26] - en_char_freq_list[char_idx])**2
        deviations.append((offset, deviation))

    offset, deviation = min(deviations, key=lambda x: x[1])
    offset_values.append(offset)

In [9]:
key = "".join(chr(ord('A')+s) for s in offset_values)
print("Key:", key)

Key: EMILY


In [10]:
clear = ""

for i in range(len(c)):
    offset = offset_values[i%key_length]
    clear += chr(((ord(c[i]) - ord('A') - offset) % 26) + ord('A'))
    
clear

'INEIGHTEENSIXTYTHREEFRIEDRICHKASISKIWASTHEFIRSTTOPUBLISHASUCCESSFULGENERALATTACKONTHEVIGENERECIPHEREARLIERATTACKSRELIEDONKNOWLEDGEOFTHEPLAINTEXTIORUSEOFARECOGNIZABLEWORDASAKEYKASISKISMETHODHADNOSUCHDEPENDENCIESKASISKIWASTHEFIRSTTOPUBLISHANACCOUNTOFTHEATTACKBUTITISCLEARTHATTHEREWEREOTHERSWHOWEREAWAREOFITINEIGHTEENFIFTYFOURCHARLESBABBAGEWASGOADEDINTOBREAKINGTHEVIGENERECIPHERWHENJOHNHALLBROCKTHWAITESSUBMITTEDANEWCIPHERTOTHEJOURNALOFTHESOCIETYOFTHEARTSWHENBABBAGESHOWEDTHATTHWAITESCIPHERWASESSENTIALLYJUSTANOTHERRECREATIONOFTHEVIGENERECIPHERTHWAITESCHALLENGEDBABBAGETOBREAKHISCIPHERENCODEDTWICEWITHKEYSOFDIFFERENTLENGTHBABBAGESUCCEEDEDINDECRYPTINGASAMPLEWHICHTURNEDOUTTOBETHEPOEMTHEVISIONOFSINBYALFREDTENNYSONENCRYPTEDACCORDINGTOTHEKEYWORDEMILYTHEFIRSTNAMEOFTENNYSONSWIFEBABBAGENEVEREXPLAINEDTHEMETHODHEUSEDSTUDIESOFBABBAGESNOTESREVEALTHATHEHADUSEDTHEMETHODLATERPUBLISHEDBYKASISKIANDSUGGESTTHATHEHADBEENUSINGTHEMETHODASEARLYASEIGHTEENFOURTYSIX'